### Imports and global utils

In [1]:
'''imports'''
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,4,5,6,7"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pickle
# clear GPU memory
from utils import general_utils, eoc
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer, T5Tokenizer
from tqdm import tqdm
from datasets import load_dataset, concatenate_datasets
from typing import Tuple, List
import torch.nn.functional as F
import eoc_datasets
from model_configs import model_configs

/home/nus-ytj/miniconda3/envs/inconsistencies/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Specify model and load tokenizer
model_identifier = "google-ul2"
# model_identifier = "t5-11b" 
# model_identifier = "flan-ul2"

config = model_configs[model_identifier]

model_name, model_dir, mode, no_extra_tokens, kwargs = \
    config['model_name'], config['model_dir'], config['mode'], config['no_extra_tokens'], config['kwargs']

# Use custom huggingface cache dirs in case the default one has low capacity, since the models are large.
MY_HUGGINGFACE_CACHE_DIR ='huggingface_cache'

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=os.path.join(MY_HUGGINGFACE_CACHE_DIR, model_dir)
)

# define loss and get extra ids
ce_loss = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id) #reduction='avg'
ce_loss_sum = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id, reduction='sum') #reduction='sum'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
RUN_CELL = True    # Load model
if RUN_CELL:
    model = T5ForConditionalGeneration.from_pretrained(
        model_name,
        cache_dir=os.path.join(MY_HUGGINGFACE_CACHE_DIR, model_dir),
        **kwargs
    )

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.84s/it]


In [5]:
subjects = ['tracking_shuffled_objects_seven_objects', 'salient_translation_error_detection', 'tracking_shuffled_objects_three_objects', 'logical_deduction_five_objects', 'hyperbaton', 'logical_deduction_seven_objects', 'logical_deduction_three_objects', 'snarks', 'disambiguation_qa', 'temporal_sequences', 'tracking_shuffled_objects_five_objects', 'penguins_in_a_table', 'date_understanding']

for subject in subjects:
    print(f"Subject: {subject}")
    dataset_processor = eoc_datasets.BigBenchProcessor(subjects=[subject])
    data = dataset_processor.get_dataset(
        set_partition='train', 
        # shuffle=True, # for hellaswag; index bias: the first 1000 examples have very low accuracy compared to the whole
        # first_k_instances=1000, # see above 
    )

    example_generator = dataset_processor.example_generator
    def tensors_filtering_criterion(input_ids, completions_batch):
        # return True
        # remove trailing padding from completions   
        # print('input_ids:', input_ids)
        # print('completions_batch:', completions_batch)
        return len(input_ids[0]) > 20 \
            and all([len(general_utils.remove_trailing_zeros_from_1d_tensor(completion)) < 5 for completion in completions_batch]) \
            # and all([len(general_utils.remove_trailing_zeros_from_1d_tensor(completion)) < 6 for completion in completions_batch]) \
            # and not all([len(general_utils.remove_trailing_zeros_from_1d_tensor(completion)) < 4 for completion in completions_batch])

    gen = example_generator(data, tokenizer, mode=mode, tensors_filtering_criterion=tensors_filtering_criterion)
    input_lens = []
    completion_lens = []
    for example_id, input_ds, completions_batch, label in gen:
        input_lens.append(len(input_ids[0]))
        completion_lens.append(len(completions_batch[0])) # with padding, this is the max len of the completions
    # print(f"input len > 20 and completion len < 10  and len > 6: {sum([i > 20 and j < 6 for i, j in zip(input_lens, completion_lens)])}")
    # print(f"completion len < 6: {sum([j < 6 for j in completion_lens])}")
    # print(f"input len max: {max(input_lens)}, min: {min(input_lens)}, avg: {sum(input_lens)/len(input_lens)}")
    # print(f"completion len max: {max(completion_lens)}, min: {min(completion_lens)}, avg: {sum(completion_lens)/len(completion_lens)}")
    baseline = dict() 
    # save the label and the number of completions
    gen = example_generator(data, tokenizer, mode, tensors_filtering_criterion=tensors_filtering_criterion)
    for example_id, input_ids, completions_batch, label in tqdm(gen):
        baseline[example_id] = dict()
        baseline[example_id]['label'] = label
        baseline[example_id]['no_completions'] = len(completions_batch)
        baseline[example_id]['p_map'] = []
        p_and_completion = []
        outputs = eoc.multi_labels_forward(model, input_ids.cuda(), completions_batch.cuda())

        for completion_index in range(len(completions_batch)):
            p = -ce_loss(
                # Only care about the tokens corresponding to the last word and omit offset tokens 
                # if the first one is <extra_id_0> and it is omitted
                outputs.logits[completion_index][no_extra_tokens:].cuda(), 
                completions_batch[completion_index][no_extra_tokens:].cuda()
            )

            baseline[example_id]['p_map'] += [p.detach().cpu().tolist()]
    '''Define the EOC function'''
    # Max reduction to emsemble conditionals for the same last word
    '''Max reduction to emsemble conditionals for the same last word, 
    i.e., only the maximum avg_log_p is kept for each last word across different range_middle_span_length's and range_middle_to_end_gap's.
    Emsemble the baseline conditionals with the K-offset conditionals and middle-off conditionals.'''

    def run_eoc(offsets, length_gap_num_tuples):
        add_baseline = True
        add_k_offset = offsets != []
        add_multispan = length_gap_num_tuples != []

        count_correct = 0
        for example_index in range(len(baseline)):
            no_completions = baseline[example_index]['no_completions']
            # Create a list of tuples (avg_log_p, completion) for each completion
            p_and_completion = []
            
            # add the baseline (offset = 0 from K-offset ensemble) to the list
            if add_baseline:
                p_and_completion += [
                    (baseline[example_index]['p_map'][completion_index], completion_index)
                    for completion_index in range(no_completions)
                ]
                
            # add the whole K-offset ensemble to the list
            if add_k_offset:
                for offset in offsets:
                    p_and_completion += [
                        (p_map_offset[(example_index, offset, completion_index)], completion_index)
                        for completion_index in range(no_completions)
                    ]
                    
            if add_multispan:
                p_and_completion += [
                    (p_map_multispan[(example_index, *length_gap_num, completion_index)], completion_index)
                    for completion_index in range(no_completions)
                    for length_gap_num in length_gap_num_tuples
                ]

            # Find the tuple with the maximum avg_log_p; this is essentially max reduction
            _, best_completion_index = max(p_and_completion, key=lambda x: x[0])
            label = baseline[example_index]['label']
            if (isinstance(label, int) and best_completion_index == label) or \
            (isinstance(label, list) and best_completion_index in label) :# TruthfulQA has multiple correct answers
                count_correct += 1
            
        # print("accuracy:", count_correct / len(baseline))
        return count_correct / len(baseline)
    from itertools import combinations
    import random        
    # K-offset conditionals
    ALL_OFFSETS = [1, 2, 3,]
    # Multispan conditionals
    ALL_LENGTH_GAP_NUM_TUPLES = [
        (3, 5, 1),
        (3, 5, 2),
        (3, 3, 1),
        (3, 3, 2),
        (3, 4, 1),
        (3, 4, 2),
    ]
    NO_OFFSETS = len(ALL_OFFSETS)
    NO_MULTISPAN = len(ALL_LENGTH_GAP_NUM_TUPLES)
    NO_DISTS_RANGE = [0]
    avg_accs = []
    for NO_DISTS in NO_DISTS_RANGE: # no of distributions to ensemble
        all_dist_ids = list(combinations(range(NO_MULTISPAN + NO_OFFSETS), NO_DISTS))
        # shuffle and take the first 100
        random.shuffle(all_dist_ids)
        all_dist_ids = all_dist_ids[:500]
        all_accs = []
        for dist_ids in all_dist_ids:
            offsets = []
            length_gap_num_tuples = []
            for dist_id in dist_ids:
                if dist_id < NO_OFFSETS:
                    offsets.append(ALL_OFFSETS[dist_id])
                else:
                    length_gap_num_tuples.append(ALL_LENGTH_GAP_NUM_TUPLES[dist_id - NO_OFFSETS])            
            acc = run_eoc(
                offsets,
                length_gap_num_tuples,
            )
            # print offsets and length_gap_num_tuples and acc
            # print(offsets, length_gap_num_tuples, acc)
            all_accs.append(acc)
        avg_acc = sum(all_accs) / len(all_accs)
        avg_accs.append(avg_acc)
        # print number of dists and avg_acc
        print(f"NO_DISTS: {NO_DISTS}, avg_acc: {avg_acc}")

Subject: tracking_shuffled_objects_seven_objects


57it [00:06, 10.75it/s]